In [3]:
import torch
import torch.nn as nn
from torch.nn import functional as F

import time

In [4]:
# hyperparameters
batch_size = 16 # how many independent sequences will we process in parallel?
block_size = 256 # what is the maximum context length for predictions?
max_iters = 20000
eval_interval = 500
learning_rate = 3e-4
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
eval_iters = 200
n_embd = 384
n_head = 24
n_layer = 24
dropout = 0.2
# ------------

cuda


In [5]:
# wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
#with open('code.txt', 'r', encoding='utf-8') as f:
#    text = f.read()
    
with open('rich_dad_poor_dad.txt', 'r', encoding='utf-8') as f:
    text = f.read()


# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

# Train and test splits
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data)) # first 90% will be train, rest va
train_data = data[:n]
val_data = data[n:]

In [6]:

# data loading
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

head_size = n_embd // n_head

global_query = nn.Linear(n_embd, head_size, bias=False)
global_value = nn.Linear(n_embd, head_size, bias=False)

class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = global_query
        self.value = global_value
        #self.query = nn.Linear(n_embd, head_size, bias=False)
        #self.value = nn.Linear(n_embd, head_size, bias=False)        
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x)   # (B,T,C)
        q = self.query(x) # (B,T,C)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * k.shape[-1]**-0.5 # (B, T, C) @ (B, C, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,C)
        out = wei @ v # (B, T, T) @ (B, T, C) -> (B, T, C)
        return out

class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

class GPTLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx


In [7]:
model = GPTLanguageModel()
#model.load_state_dict(torch.load("./model_code"))
#model.load_state_dict(torch.load("./model_rich"))

m = model.to(device)
# print the number of parameters in the model
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

# 1, 35.664477 M parameters
# 2, 42.730077 M parameters

35.664477 M parameters


In [8]:
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

In [9]:

start_time = time.time()  # Record the start time
for iter in range(5000):
    
    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")
        end_time = time.time()  # Record the end time
        time_taken = end_time - start_time  # Calculate time taken

        print(f"Iteration {iter} took {time_taken:.4f} seconds")  # Print the time taken

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

step 0: train loss 4.7303, val loss 4.7276
Iteration 0 took 62.8014 seconds
step 500: train loss 2.1050, val loss 2.1421
Iteration 500 took 419.9329 seconds
step 1000: train loss 1.4377, val loss 1.5771
Iteration 1000 took 794.4735 seconds
step 1500: train loss 1.1789, val loss 1.4344
Iteration 1500 took 1176.9022 seconds
step 2000: train loss 0.9970, val loss 1.4013
Iteration 2000 took 1562.0558 seconds
step 2500: train loss 0.8500, val loss 1.4090
Iteration 2500 took 1943.2764 seconds
step 3000: train loss 0.7051, val loss 1.4867
Iteration 3000 took 2325.5763 seconds
step 3500: train loss 0.5735, val loss 1.5514
Iteration 3500 took 2712.5934 seconds
step 4000: train loss 0.4482, val loss 1.6840
Iteration 4000 took 3122.4039 seconds
step 4500: train loss 0.3458, val loss 1.7912
Iteration 4500 took 3601.7585 seconds


In [10]:
# 1 single Q and V
'''
step 0: train loss 4.7373, val loss 4.7395
Iteration 0 took 65.3062 seconds
step 500: train loss 2.1451, val loss 2.1847
Iteration 500 took 446.0307 seconds
step 1000: train loss 1.4346, val loss 1.5838
Iteration 1000 took 830.1166 seconds
step 1500: train loss 1.1743, val loss 1.4413
Iteration 1500 took 1212.6841 seconds
step 2000: train loss 1.0143, val loss 1.4061
Iteration 2000 took 1592.8847 seconds
step 2500: train loss 0.8537, val loss 1.4149
Iteration 2500 took 1973.1572 seconds
step 3000: train loss 0.7194, val loss 1.4786
Iteration 3000 took 2351.7374 seconds
step 3500: train loss 0.5820, val loss 1.5589
Iteration 3500 took 2731.0541 seconds
step 4000: train loss 0.4605, val loss 1.6484
Iteration 4000 took 3110.3526 seconds
step 4500: train loss 0.3581, val loss 1.7627
Iteration 4500 took 3490.8806 seconds
step 5000: train loss 0.2721, val loss 1.8673
Iteration 5000 took 3867.1339 seconds
step 5500: train loss 0.1988, val loss 2.0157
'''

'\nstep 0: train loss 4.7373, val loss 4.7395\nIteration 0 took 65.3062 seconds\nstep 500: train loss 2.1451, val loss 2.1847\nIteration 500 took 446.0307 seconds\nstep 1000: train loss 1.4346, val loss 1.5838\nIteration 1000 took 830.1166 seconds\nstep 1500: train loss 1.1743, val loss 1.4413\nIteration 1500 took 1212.6841 seconds\nstep 2000: train loss 1.0143, val loss 1.4061\nIteration 2000 took 1592.8847 seconds\nstep 2500: train loss 0.8537, val loss 1.4149\nIteration 2500 took 1973.1572 seconds\nstep 3000: train loss 0.7194, val loss 1.4786\nIteration 3000 took 2351.7374 seconds\nstep 3500: train loss 0.5820, val loss 1.5589\nIteration 3500 took 2731.0541 seconds\nstep 4000: train loss 0.4605, val loss 1.6484\nIteration 4000 took 3110.3526 seconds\nstep 4500: train loss 0.3581, val loss 1.7627\nIteration 4500 took 3490.8806 seconds\nstep 5000: train loss 0.2721, val loss 1.8673\nIteration 5000 took 3867.1339 seconds\nstep 5500: train loss 0.1988, val loss 2.0157\n'

In [11]:
# 2,  multiple Q and V
'''
step 0: train loss 4.6681, val loss 4.6663
Iteration 0 took 63.5645 seconds
step 500: train loss 2.3191, val loss 2.3342
Iteration 500 took 423.8687 seconds
step 1000: train loss 1.5196, val loss 1.6449
Iteration 1000 took 805.0172 seconds
step 1500: train loss 1.1968, val loss 1.4544
Iteration 1500 took 1184.8161 seconds
step 2000: train loss 0.9982, val loss 1.4144
Iteration 2000 took 1560.8502 seconds
step 2500: train loss 0.8258, val loss 1.4542
Iteration 2500 took 1939.0441 seconds
step 3000: train loss 0.6541, val loss 1.5132
Iteration 3000 took 2321.2756 seconds
step 3500: train loss 0.4890, val loss 1.6229
Iteration 3500 took 2707.2526 seconds
step 4000: train loss 0.3466, val loss 1.7947
Iteration 4000 took 3094.3183 seconds
step 4500: train loss 0.2485, val loss 1.9147
Iteration 4500 took 3477.0767 seconds
'''

'\nstep 0: train loss 4.6681, val loss 4.6663\nIteration 0 took 63.5645 seconds\nstep 500: train loss 2.3191, val loss 2.3342\nIteration 500 took 423.8687 seconds\nstep 1000: train loss 1.5196, val loss 1.6449\nIteration 1000 took 805.0172 seconds\nstep 1500: train loss 1.1968, val loss 1.4544\nIteration 1500 took 1184.8161 seconds\nstep 2000: train loss 0.9982, val loss 1.4144\nIteration 2000 took 1560.8502 seconds\nstep 2500: train loss 0.8258, val loss 1.4542\nIteration 2500 took 1939.0441 seconds\nstep 3000: train loss 0.6541, val loss 1.5132\nIteration 3000 took 2321.2756 seconds\nstep 3500: train loss 0.4890, val loss 1.6229\nIteration 3500 took 2707.2526 seconds\nstep 4000: train loss 0.3466, val loss 1.7947\nIteration 4000 took 3094.3183 seconds\nstep 4500: train loss 0.2485, val loss 1.9147\nIteration 4500 took 3477.0767 seconds\n'

In [12]:
start_time = time.time()  # Record the start time

text = "“The truth about what?” I asked."
context = torch.tensor([encode(text)], dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=200)[0].tolist()))

end_time = time.time()  # Record the end time
time_taken = end_time - start_time  # Calculate time taken

print(f"Iteration {iter} took {time_taken:.4f} seconds")  # Print the time taken, 
# 1. took 33.3500 seconds
# 2. 33.3966 seconds

“The truth about what?” I asked. 

our financial cople earn that something reasons, but the fear of doing the ight themselves of 
business or ctable sheetelves a broke, and there in bent out people from of foten the door, because 
a
Iteration 4999 took 77.5299 seconds


In [13]:
#torch.save(m.state_dict(), "./model_code")
torch.save(m.state_dict(), "./model_rich")

In [14]:
start_time = time.time()  # Record the start time

text = "How do I get rich"
context = torch.tensor([encode(text)], dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=1000)[0].tolist()))

end_time = time.time()  # Record the end time
time_taken = end_time - start_time  # Calculate time taken

print(f"Iteration {iter} took {time_taken:.4f} seconds")  # Print the time taken

# 3. took 165.9984 seconds

How do I get rich quicked and with anger and head of the counntry. Soons after they are 
leader, they better again, and interesting areallyzerge. And the merize was laws shying. 

Many empired and Alerge was he got bring sets column. The sats are the cources even 
when it wages mostly are estated to being good doors exciting from greatenters, my most wife parents 
are goup because the money and investment. They car not tellicy the poor and 
middle class tente and me. 

Our will heaving dreams over greater deals on financial intellligence how thinking would geney. His 
focused it benefits by angring patcher was dear. The other hander was need to both 
and the veroxing potent. 

I looked to understand why I love worked for the road table. 

1. 

No 4. I Even se a hourt again it was designed and a pension. Remember Badly was able to 
tea financial expense. It took his promotion and member when it was a rich man and just for 
the Teconomic of Education conninerding ’being an intellectual, 